In [95]:
import yaml
def read_db_creds(file_path='/Users/yasir/Documents/Multinational_Retail_Data/Multinational_Retail_Data/db_creds.yaml'):
    with open(file_path,'r') as file:
        creds = yaml.safe_load(file)
        return creds 

In [96]:
read_db_creds()

{'RDS_HOST': 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com',
 'RDS_PASSWORD': 'AiCore2022',
 'RDS_USER': 'aicore_admin',
 'RDS_DATABASE': 'postgres',
 'RDS_PORT': 5432}

In [97]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [98]:
import yaml
from sqlalchemy import inspect
from sqlalchemy import create_engine

class DatabaseConnector:
    def __init__(self):
        self.file_path = '/Users/yasir/Documents/Multinational_Retail_Data/Multinational_Retail_Data/db_creds.yaml'

    def read_db_creds(self):
        with open(self.file_path,'r') as file:
            creds = yaml.safe_load(file)
            return creds 
    
    def init_db_engine(self):
        creds = self.read_db_creds()
        db_url = f"{'postgresql'}+{'psycopg2'}://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}"
        engine = create_engine(db_url)
        return engine
    
    def list_db_tables(self, engine):
        inspector = inspect(engine)
        return inspector.get_table_names()

In [99]:
db_connector = DatabaseConnector()
db_engine = db_connector.init_db_engine()


In [100]:
tables_list = db_connector.list_db_tables(db_engine)
tables_list

['legacy_store_details', 'legacy_users', 'orders_table']

# DataExtractor.py

In [17]:
pip install pandas requests boto3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
table_name = tables_list[1]

In [12]:
import pandas as pd


class DataExtractor:

    def __init__(self):
        pass

    def read_rds_table(self, db_connector, table_name):
        engine = db_connector.init_db_engine()
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        return df  

In [32]:
df = DataExtractor().read_rds_table(db_connector,table_name)



In [33]:
df

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [34]:
df.columns

Index(['index', 'first_name', 'last_name', 'date_of_birth', 'company',
       'email_address', 'address', 'country', 'country_code', 'phone_number',
       'join_date', 'user_uuid'],
      dtype='object')

In [53]:
df[df['first_name'] == 'NULL'].count()

index            21
first_name       21
last_name        21
date_of_birth    21
company          21
email_address    21
address          21
country          21
country_code     21
phone_number     21
join_date        21
user_uuid        21
dtype: int64

In [47]:
df.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

DataCleaning.py

In [91]:
class DataCleaning:

    def clean_user_data(self, df):

        df = self.remove_null_values(df)
        df = self.clean_invalid_dates(df,'date_of_birth')
        df = self.clean_invalid_dates(df,"join_date")
        df.drop(columns='index',inplace=True)
        return df
    
    def remove_null_values(self, df):
        return df[~df.applymap(lambda x: 'NULL' in str(x)).any(axis=1)]

    def clean_invalid_dates(self,df,column_name):
        df[column_name] = pd.to_datetime(df[column_name], format='%Y %B %d', errors='ignore')
        return df
        
    

In [93]:
data_cleaner = DataCleaning()
data_cleaned_bharatan_spartan = data_cleaner.clean_user_data(df)
data_cleaned_bharatan_spartan


/var/folders/25/2c9g2pgs07x32rvtdn4fcld00000gn/T/ipykernel_7293/3397019056.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df[~df.applymap(lambda x: 'NULL' in str(x)).any(axis=1)]
/var/folders/25/2c9g2pgs07x32rvtdn4fcld00000gn/T/ipykernel_7293/3397019056.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_datetime(df[column_name], format='%Y %B %d', errors='ignore')
/var/folders/25/2c9g2pgs07x32rvtdn4fcld00000gn/T/ipykernel_7293/3397019056.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...
15315,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [90]:
data_cleaned_bharatan_spartan

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [58]:
test = filtered_rows[filtered_rows['date_of_birth'].str.contains('October')]
test['date_of_birth'] = pd.to_datetime(test['date_of_birth'])
test

/var/folders/25/2c9g2pgs07x32rvtdn4fcld00000gn/T/ipykernel_7293/614364784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date_of_birth'] = pd.to_datetime(test['date_of_birth'])


,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
360,360,Margaret,Akhtar,1968-10-16,"Turner, Marshall and Clarke",tcoleman@saunders-blake.org,033 Poole gateway\nLake Sharon\nCW62 6SD,United Kingdom,GB,(0191) 496 0493,1994-11-21,90f1f4cf-aca3-4c23-bc74-cc9f8e58c5ae
3066,3069,Lioba,Dobes,1946-10-18,Heinrich Stiftung & Co. KG,hthanel@lehmann.de,Rosestr. 6\n97463 Griesbach Rottal,Germany,DE,+49(0)0334 246986,1993-07-22,a272077a-e52a-48e4-a213-c93227f34d84
7259,7267,Holly,Thomas,1948-10-24,Whittaker-Baker,derek65@middleton.com,53 Craig island\nSamanthafort\nG16 3JJ,United Kingdom,GB,+449098790279,2020-05-20,75c54a4b-a143-434a-92b1-fb780d154662


In [35]:
cleaned_data = DataCleaning()
cleaned_df = cleaned_data.clean_user_data(df)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df